# SWOT L4 - June 1, 2024

0.125° resolution, MIOST interpolation

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path

## Load data

In [ ]:
ds = xr.open_dataset('swot_l4_data/dt_global_allsat_phy_l4_20240601_20250331.nc')
ds

In [ ]:
for var in ds.data_vars:
    print(f"{var}: {ds[var].attrs.get('long_name', '')}")

In [ ]:
sla = ds['sla'].isel(time=0)
adt = ds['adt'].isel(time=0)
ugos = ds['ugos'].isel(time=0)
vgos = ds['vgos'].isel(time=0)
speed = np.sqrt(ugos**2 + vgos**2)

## Global plot

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = plt.subplot(1, 1, 1, projection=ccrs.Robinson())

im = sla.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdBu_r',
              vmin=-0.4, vmax=0.4, add_colorbar=False)

ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.7)
cbar.set_label('SLA (m)', fontsize=12)

ax.set_title('SWOT L4 - June 1, 2024', fontsize=14)
plt.tight_layout()
plt.savefig('swot_l4_global.png', dpi=150)
plt.show()

## Multi-panel

In [ ]:
fig = plt.figure(figsize=(20, 14))

# Global SLA
ax1 = plt.subplot(2, 3, 1, projection=ccrs.Robinson())
sla.plot(ax=ax1, transform=ccrs.PlateCarree(), cmap='RdBu_r', 
         vmin=-0.4, vmax=0.4, add_colorbar=True, cbar_kwargs={'shrink': 0.7})
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax1.set_title('SLA', fontsize=12)

# Pacific
ax2 = plt.subplot(2, 3, 2, projection=ccrs.PlateCarree(central_longitude=180))
sla.plot(ax=ax2, transform=ccrs.PlateCarree(), cmap='RdBu_r',
         vmin=-0.4, vmax=0.4, add_colorbar=True, cbar_kwargs={'shrink': 0.7})
ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax2.set_extent([120, 240, -60, 60], crs=ccrs.PlateCarree())
ax2.set_title('Pacific', fontsize=12)

# Atlantic
ax3 = plt.subplot(2, 3, 3, projection=ccrs.PlateCarree())
sla.plot(ax=ax3, transform=ccrs.PlateCarree(), cmap='RdBu_r',
         vmin=-0.4, vmax=0.4, add_colorbar=True, cbar_kwargs={'shrink': 0.7})
ax3.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax3.set_extent([-80, 20, -60, 60], crs=ccrs.PlateCarree())
ax3.set_title('Atlantic', fontsize=12)

# ADT
ax4 = plt.subplot(2, 3, 4, projection=ccrs.Robinson())
adt.plot(ax=ax4, transform=ccrs.PlateCarree(), cmap='viridis',
         add_colorbar=True, robust=True, cbar_kwargs={'shrink': 0.7})
ax4.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax4.set_title('ADT', fontsize=12)

# Current speed
ax5 = plt.subplot(2, 3, 5, projection=ccrs.Robinson())
speed.plot(ax=ax5, transform=ccrs.PlateCarree(), cmap='YlOrRd',
          vmin=0, vmax=1.0, add_colorbar=True, cbar_kwargs={'shrink': 0.7})
ax5.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax5.set_title('Current Speed', fontsize=12)

# Histogram
ax6 = plt.subplot(2, 3, 6)
sla_flat = sla.values.flatten()
sla_flat = sla_flat[~np.isnan(sla_flat)]
ax6.hist(sla_flat, bins=100, color='steelblue', alpha=0.7)
ax6.set_xlabel('SLA (m)')
ax6.set_ylabel('Count')
ax6.set_title('Distribution', fontsize=12)

plt.tight_layout()
plt.savefig('swot_l4_multi.png', dpi=150)
plt.show()

## Regional

In [ ]:
regions = [
    {'name': 'NE Pacific', 'extent': [-180, -120, 40, 65]},
    {'name': 'West Coast', 'extent': [-135, -115, 30, 50]},
    {'name': 'Gulf of Mexico', 'extent': [-100, -80, 18, 32]},
    {'name': 'Gulf Stream', 'extent': [-80, -60, 30, 45]}
]

fig, axes = plt.subplots(2, 2, figsize=(16, 12), 
                         subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

for ax, region in zip(axes, regions):
    sla.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdBu_r',
             vmin=-0.4, vmax=0.4, add_colorbar=True,
             cbar_kwargs={'shrink': 0.7, 'orientation': 'horizontal'})
    
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_extent(region['extent'], crs=ccrs.PlateCarree())
    ax.set_title(region['name'], fontsize=12)

plt.tight_layout()
plt.savefig('swot_l4_regional.png', dpi=150)
plt.show()

## Extract subset

In [ ]:
regional_ds = ds.sel(longitude=slice(-180, -120), latitude=slice(40, 65))
regional_ds.to_netcdf('swot_l4_ne_pacific.nc')
regional_ds['sla'].isel(time=0).shape

In [ ]:
ds.close()